In [ ]:
import os
import json
import logging
import sys
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
from collections import defaultdict
import uuid
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from tenacity import retry, stop_after_attempt, wait_fixed



In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

In [ ]:

# Azure OpenAI Configuration
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "",
            "base_url": "",
            "api_type": "azure",
            "api_version": "2024-02-01"
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42
}

In [5]:
# Create directories for data storage
os.makedirs("data/conversation_history", exist_ok=True)
os.makedirs("data/progress_data", exist_ok=True)
os.makedirs("data/student_conversations", exist_ok=True)

In [ ]:

# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent. Explain concepts clearly, adapt to the student's level, provide step-by-step breakdowns, and encourage critical thinking. Offer examples and practice questions. Respond in a supportive, engaging tone."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a high-school Student Agent. Actively engage by:
- Attempting to solve problems and sharing your steps (e.g., 'I factored x^2 - 7x + 10 as (x-5)(x-2)').
- Asking questions if unsure (e.g., 'Can you explain the quadratic formula?').
- Reflecting on learning (e.g., 'I get factoring but struggle with graphing').
- Responding to tutor prompts with effort, even if incomplete."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Analyze student performance from ProgressMemory, generate detailed progress reports, and suggest focus areas. Provide reports in a student-friendly JSON format."""

In [ ]:


# Conversation Memory
class EnhancedConversationMemory:
    def __init__(self, session_id: Optional[str] = None, storage_path: str = "data/conversation_history"):
        self.session_id = session_id or str(uuid.uuid4())
        self.storage_path = storage_path
        self.conversation_history: List[Dict[str, Any]] = []
        os.makedirs(self.storage_path, exist_ok=True)

    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, Any]] = None):
        message = {"role": role, "content": content, "timestamp": datetime.now().isoformat()}
        self.conversation_history.append(message)

    def get_messages(self) -> List[Dict[str, Any]]:
        return self.conversation_history

In [ ]:
# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent. Explain concepts clearly, adapt to high-school level, provide step-by-step breakdowns, and include examples and practice questions. Evaluate student responses, offer feedback, and maintain a supportive tone."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a high-school Student Agent. Engage actively by:
- Attempting to solve problems and showing your steps (e.g., 'I factored x^2 - 7x + 10 as (x-5)(x-2)').
- Asking clarifying questions if unsure (e.g., 'Can you explain the quadratic formula?').
- Reflecting on your learning (e.g., 'I understand factoring but need help with graphing').
- Responding to tutor prompts with effort, even if unsure (e.g., 'I’ll try: ...')."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Use ProgressMemory data to generate student-friendly JSON progress reports. Include a summary, strengths, areas for improvement, and recommendations. Handle cases with minimal data gracefully."""

In [ ]:
# Conversation Memory
class EnhancedConversationMemory:
    def __init__(self, session_id: Optional[str] = None, storage_path: str = "data/conversation_history"):
        self.session_id = session_id or str(uuid.uuid4())
        self.storage_path = storage_path
        self.conversation_history: List[Dict[str, Any]] = []
        os.makedirs(self.storage_path, exist_ok=True)

    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, str]] = None):
        message = {
            "role": content,
            "timestamp": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        self.conversation_history.append({"role": role, "content": content})
        try:
            with open(os.path.join(self.storage_path, f"{self.session_id}.json"), "w") as f:
                json.dump({"history": self.conversation_history}, f, indent=2)
        except Exception as e:
            logger.error(f"Failed to save conversation: {e}")

    def get_messages(self) -> List[Dict[str, Any]]:
        return self.conversation_history

In [ ]:

# Progress Memory
@dataclass
class LearningProgress:
    subject: str
    topic: str
    skill_level: float
    attempts: int
    successful_attempts: int

@dataclass
class LearningSession:
    session_id: str
    start_time: str
    subjects_covered: List[str]
    questions_asked: int
    questions_answered_correctly: int

class ProgressMemory:
    def __init__(self, student_id: str, storage_path: str = "data/progress_data"):
        self.student_id = student_id
        self.storage_path = storage_path
        self.progress_data: Dict[str, Dict[str, LearningProgress]] = defaultdict(dict)
        self.learning_sessions: List[LearningSession] = []
        os.makedirs(self.storage_path, exist_ok=True)

    def start_session(self, session_id: str) -> LearningSession:
        session = LearningSession(
            session_id=session_id,
            start_time=datetime.now().isoformat(),
            subjects_covered=[],
            questions_asked=0,
            questions_answered_correctly=0
        )
        self.learning_sessions.append(session)
        return session

    def update_progress(self, subject: str, topic: str, performance_score: float, was_successful: bool):
        if subject not in self.progress_data:
            self.progress_data[subject] = {}
        if topic not in self.progress_data[subject]:
            self.progress_data[subject][topic] = LearningProgress(subject, topic, 0.0, 0, 0)
        progress = self.progress_data[subject][topic]
        progress.attempts += 1
        if was_successful:
            progress.successful_attempts += 1
        progress.skill_level = (progress.skill_level * 0.7) + (performance_score * 0.3)
        logger.info(f"Updated progress: {subject}/{topic}, skill={progress.skill_level}")

    def update_session(self, session_id: str, subject: str, topic: str, question_asked: bool = False, correct_answer: bool = False):
        for session in self.learning_sessions:
            if session.session_id == session_id:
                if subject not in session.subjects_covered:
                    session.subjects_covered.append(subject)
                if question_asked:
                    session.questions_asked += 1
                if correct_answer:
                    session.questions_answered_correctly += 1
                logger.info(f"Updated session {session_id}: subject={subject}, questions_asked={session.questions_asked}")
                break

    def get_progress_report(self) -> Dict[str, Any]:
        return {
            "student_id": self.student_id,
            "subjects": {s: {t: asdict(p) for t, p in topics.items()} for s, topics in self.progress_data.items()},
            "sessions": [asdict(s) for s in self.learning_sessions]
        }



In [ ]:
# Create Agents
def create_educational_agents(student_id: str, session_id: str):
    progress_memory = ProgressMemory(student_id)
    conv_memory = EnhancedConversationMemory(session_id)
    progress_memory.start_session(session_id)

    tutor = AssistantAgent(
        name="Educational_Tutor",
        system_message=AgentConfig.get_tutor_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER",
        max_consecutive_auto_reply=5
    )

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def explain_concept(subject: str, topic: str, difficulty_level: str = "medium", learning_style: str = "visual"):
        prompt = f"Explain {topic} in {subject} at {difficulty_level} difficulty for a high-school student, using a {learning_style} learning style. Provide a step-by-step breakdown, one example, and one practice question."
        response = tutor.generate_reply([{"content": prompt, "role": "user"}])
        conv_memory.add_message("tutor", response, {"subject": subject, "topic": topic})
        progress_memory.update_progress(subject, topic, 0.6, True)
        progress_memory.update_session(session_id, subject, topic, question_asked=True)
        return response

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def create_practice_problems(subject: str, topic: str, count: int = 3, difficulty: str = "medium") -> Dict[str, Any]:
        prompt = f"""Generate {count} practice problems for {topic} in {subject} at {difficulty} difficulty. Include:
- One factoring problem
- One quadratic formula problem
- One word problem
Format as JSON with question, correct_answer, and solution."""
        response = tutor.generate_reply([{"content": prompt, "role": "user"}])
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        problems = json.loads(response[json_start:json_end]) if json_start != -1 else {"error": "Invalid response"}
        conv_memory.add_message("tutor", json.dumps(problems, indent=2), {"subject": subject, "topic": topic})
        progress_memory.update_session(session_id, subject, topic, question_asked=True)
        return problems

    tutor.register_function({
        "explain_concept": explain_concept,
        "create_practice_problems": create_practice_problems
    })

    student = UserProxyAgent(
        name="Student_Learner",
        system_message=AgentConfig.get_student_system_message(),
        human_input_mode="NEVER",
        max_consecutive_auto_reply=3,
        code_execution_config=False
    )

    progress_tracker = AssistantAgent(
        name="Progress_Tracker",
        system_message=AgentConfig.get_progress_tracker_system_message(),
        llm_config=llm_config,
        human_input_mode="NEVER"
    )

    @retry(stop=stop_after_attempt(3), wait=wait_fixed(5))
    def generate_progress_report():
        report = progress_memory.get_progress_report()
        if not report["subjects"] and not report["sessions"]:
            return {
                "summary": "No progress data yet. Try more activities!",
                "strengths": [],
                "areas_for_improvement": [],
                "recommendations": ["Solve practice problems", "Ask questions"],
                "timestamp": datetime.now().isoformat()
            }
        prompt = f"""Generate a JSON progress report based on:
{json.dumps(report, indent=2)}
Include:
- Summary of progress
- Strengths (skill_level > 0.7)
- Areas for improvement (skill_level < 0.6)
- Recommendations"""
        response = progress_tracker.generate_reply([{"content": prompt, "role": "user"}])
        json_start = response.find('{')
        json_end = response.rfind('}') + 1
        return json.loads(response[json_start:json_end]) if json_start != -1 else {"error": "Invalid report"}

    progress_tracker.register_function({"generate_progress_report": generate_progress_report})

    def custom_speaker_selection(last_speaker, groupchat):
        agents = groupchat.agents
        last_message = groupchat.messages[-1]["content"].strip() if groupchat.messages else ""
        if last_speaker == student and not last_message:
            return tutor  # Skip empty student responses
        return agents[(agents.index(last_speaker) + 1) % len(agents)]

    groupchat = GroupChat(
        agents=[student, tutor, progress_tracker],
        messages=[],
        max_round=20,
        speaker_selection_method=custom_speaker_selection
    )

    manager = GroupChatManager(
        groupchat=groupchat,
        llm_config=llm_config,
        name="Education_Manager"
    )

    return {
        "student": student,
        "tutor": tutor,
        "progress_tracker": progress_tracker,
        "manager": manager,
        "progress_memory": progress_memory,
        "conv_memory": conv_memory
    }

In [ ]:

def main():
    print("=== Starting Educational Session ===")
    student_id = "test_student_001"
    session_id = str(uuid.uuid4())

    try:
        agents = create_educational_agents(student_id, session_id)
        student = agents["student"]
        manager = agents["manager"]
        progress_tracker = agents["progress_tracker"]
        progress_memory = agents["progress_memory"]
        conv_memory = agents["conv_memory"]

        # Explain quadratic equations
        print("\n=== Explaining Quadratic Equations ===")
        student.initiate_chat(
            manager,
            message="Please explain quadratic equations to me at a medium difficulty level with visual learning style."
        )
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.6, True)

        # Real-world example
        print("\n=== Real-World Example ===")
        student.send(
            message="Can you show a real-world example of quadratic equations?",
            recipient=manager
        )
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.7, True)

        # Practice problems
        print("\n=== Creating Practice Problems ===")
        student.send(
            message="Could you create 3 practice problems on quadratic equations at medium difficulty?",
            recipient=manager
        )
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True)

        # Student attempts problem
        print("\n=== Student Attempting Problem ===")
        student.send(
            message="For x^2 - 7x + 10 = 0, I tried factoring. I got x = 2 and x = 5. Is this correct?",
            recipient=manager
        )
        progress_memory.update_progress("Mathematics", "Quadratic Equations", 0.8, True)
        progress_memory.update_session(session_id, "Mathematics", "Quadratic Equations", question_asked=True, correct_answer=True)

        # Generate progress report
        print("\n=== Generating Progress Report ===")
        response = student.initiate_chat(
            progress_tracker,
            message="Please generate a progress report."
        )
        report = None
        for msg in response.chat_history:
            content = msg.get("content", "")
            if isinstance(content, str) and "summary" in content:
                json_start = content.find('{')
                json_end = content.rfind('}') + 1
                if json_start != -1 and json_end != 0:
                    report = json.loads(content[json_start:json_end])
                    break
        if report:
            print(json.dumps(report, indent=2))
        else:
            print("Failed to generate progress report.")

        print("\n=== Session Statistics ===")
        final_report = progress_memory.get_progress_report()
        print(f"Total Sessions: {len(final_report['sessions'])}")
        print(f"Total Questions: {sum(s['questions_asked'] for s in final_report['sessions'])}")
        print(f"Subjects Covered: {[s for s in final_report['subjects'].keys()]}")

    except Exception as e:
        logger.error(f"Error in main: {e}")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

    print("\n=== Educational Session Completed ===")

In [ ]:


if __name__ == "__main__":
    os.environ["AUTOGEN_USE_DOCKER"] = "False"
    main()

=== Starting Educational Session ===

=== Explaining Quadratic Equations ===
Student_Learner (to Education_Manager):

Please explain quadratic equations to me at a medium difficulty level with visual learning style.

--------------------------------------------------------------------------------

Next speaker: Educational_Tutor

INFO:httpx:HTTP Request: POST https://idkrag.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
Educational_Tutor (to Education_Manager):

Certainly! Quadratic equations are a fundamental concept in algebra, and I’ll explain them step by step using visual cues and examples that will help you make sense of them.

---

### **What is a Quadratic Equation?**
A quadratic equation is a mathematical expression involving a variable (like \(x\)) raised to the second power (\(x^2\)). It follows the general form:

\[
ax^2 + bx + c = 0
\]

- \(a\), \(b\), and \(c\) are constants, and \(a \neq 0\).
- \(x\) is the variable w